# Task A: Deep Learning for ECG Heartbeat Classification

Develop  a  deep  learning  model  that  can  accurately  classify  heartbeats  from  ECG  signals.  You  are 
encouraged  to  explore  different  neural  network  architectures  and  techniques,  going  beyond  simple 
feedforward networks. 

In [1]:
import torch
print("PyTorch version:", torch.__version__)

# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())

# Get current CUDA device index (if available)
if torch.cuda.is_available():
    print("Current CUDA device index:", torch.cuda.current_device())
    print("CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("No CUDA devices found.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PyTorch version: 2.5.0+cu118
CUDA available: True
Current CUDA device index: 0
CUDA device name: NVIDIA GeForce RTX 3070


In [2]:
import pandas as pd
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import os
dataframes = {}
directory_path = 'Arrhythmia_Dataset'
all_files = os.listdir(directory_path)

for file in os.listdir(directory_path):
    if file.endswith('.csv'):
        file_path = os.path.join(directory_path, file)
        # Remove the .csv extension for the DataFrame name
        df_name = os.path.splitext(file)[0]
        dataframes[df_name] = pd.read_csv(file_path)


C:\Users\randa\AppData\Local\Temp\ipykernel_43900\1362218957.py:18: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes[df_name] = pd.read_csv(file_path)


In [3]:
print(dataframes.keys())


dict_keys(['INCART 2-lead Arrhythmia Database', 'MIT-BIH Arrhythmia Database', 'MIT-BIH Supraventricular Arrhythmia Database', 'Sudden Cardiac Death Holter Database'])


In [4]:
data_df = dataframes['INCART 2-lead Arrhythmia Database']

In [5]:
x_data = data_df.iloc[:, 2:]
y_label = data_df[['type']]

In [6]:
y_label.value_counts()

type
N       153546
VEB      20000
SVEB      1958
F          219
Q            6
Name: count, dtype: int64

# Loading model

In [7]:
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from common_utils import CNN1D


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_label)  # Encode labels without replacement

model_path = "./model/cnn_model.pth"
num_classes = len(label_encoder.classes_)
loaded_model = CNN1D(num_classes).to(device)  # Reinitialize model with the same architecture
loaded_model.load_state_dict(torch.load(model_path, weights_only=True))
print("Model loaded successfully.")

c:\Users\randa\OneDrive\Documents\GitHub\Neural-Networks-Project\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model loaded successfully.


In [ ]:
# Assuming you have a DataLoader for your test dataset
y = label_encoder.fit_transform(y_label.values.ravel())
y = np.eye(num_classes)[y] 

X = x_data
X_test = np.expand_dims(X, axis=1)    # Change axis from 2 to 1
X_test_tensor = torch.tensor(X_test).float()    # Ensure the data is of float type
y_test_tensor = torch.tensor(y).long()   

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

loaded_model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    correct_test = 0
    total_test = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move to GPU if available
        outputs = loaded_model(inputs)  # Forward pass
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels.argmax(dim=1)).sum().item()

    accuracy = correct_test / total_test  # Calculate test accuracy
    print(f'Test Accuracy: {accuracy:.4f}')


, Test Accuracy: 0.8817
